### Пример генерации отчетов по прошедшей лекции на основе комментариев студентов
Данный ноутбук содержит запуск модели saiga-llama3-8B для генерации сводных отчетов по каждой лекции. 

**Ноутбук приложен отдельно, т.к. требует достаточно больших мощностей для быстрого выполнения и не предназначен для запуска в той среде, где мы запустили модель для всех (там ЦПУ 2 ядра, оно просто умрет)** 

Решение будет протестировано на одной из лекций Geekbrains, дабы доказать свою полную готовность к работе.

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
!pip install langchain accelerate bitsandbytes>0.37.0 -q

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 23.8.0 requires cubinlinker, which is not installed.
cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
cudf 23.8.0 requires ptxcompiler, which is not installed.
cuml 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
dask-cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
keras-cv 0.8.2 requires keras-core, which is not installed.
keras-nlp 0.8.2 requires keras-core, which is not installed.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.8 which is incompatible.
apache-beam 2.46.0 requires numpy<1.25.0,>=1.14.3, but you have numpy 1.26.4 which is incompatible.
apache-beam 2.46.0 requires pyarrow<10.0.0,>=3.0.0, but you have pyarrow 15.0.2 which is incompatible.
cudf

In [3]:
from langchain.llms.huggingface_pipeline import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema.chat_history import BaseChatMessageHistory
from langchain.text_splitter import RecursiveCharacterTextSplitter
from transformers import AutoTokenizer
import torch

import os
import json
import pandas as pd

Для запуска этой модели мы используем GPU P100. В данный ускоритель выбранная модель не влезает без квантизации в 8 бит. Если у вас есть видеокарта с объемом памяти ~24 гигабайта (4090, V100, A100, etc.), то для вас будет ниже приложен конфиг, который может обеспечить лучшее качество генерации.

In [4]:
better_model_config = {
    'top_p': 0.9,
    "temperature": 0.6,
    "max_length": 1536,
    "do_sample": True
}

# в зависимости от версии вашей видеокарты, можно добавить параметр 'attn_implementation' для ускорения модели

**НЕ СТАВЬТЕ** параметр 'top_p' одновременно с квантизацией модели - работать не будет))

In [5]:
gpu_llm = HuggingFacePipeline.from_model_id(
    model_id="IlyaGusev/saiga_llama3_8b",
    task="text-generation",
    device=0,
    batch_size=2,
    model_kwargs={
        'load_in_8bit': True,
        "temperature": 0.6,
        "max_length": 8192,
        "do_sample": True
    },
)

2024-04-28 07:43:32.974440: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-28 07:43:32.974576: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-28 07:43:33.097826: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/446 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


...

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/277 [00:00<?, ?B/s]

In [7]:
EXTRACT_INTRO_PROMPT = '''
Тебе будет дан текст, в котором будут даны комментарии к лекции для студентов. Комментарии могут быть на любую тему, но все они были написаны именно учениками и во время пары с преподавателем..
Проанализируй эти комментарии и выдели в них основные моменты, описанные ниже. 
Тебе необходимо выделить:
1. Претензии от студентов к самой лекции (к качеству звука, качеству материала и любым другим параметрам)
2. Что в лекции понравилось студентам;
3. На какую тему преимущественно задавались вопросы.
Отвечай только на русском языке. 
В самом конце сделай подведение итогов, исходя из всех комментариев, что тебе были даны. Если корректно выполнишь задачу и будешь следовать условию, которое тебе было задано, я дам тебе чаевые в размере $20.

Текст:
BEGIN_TEXT
%s
END_TEXT
Ответ:
'''

In [8]:
BASE_PROMPT = '''
{prompt}
'''

In [9]:
df = pd.read_csv('/kaggle/input/cfo-file/vlad.csv')
df.head(3)

,Unnamed: 0,ID урока,Дата старта урока,Роль пользователя,Текст сообщения,Дата сообщения,Разметка
0,118188,423515.0,2024-03-03 09:49:53,user,сегодня значит быстрая сортировка,2024-03-03 09:58:45,NaN
1,118189,423515.0,2024-03-03 09:49:53,user,"быстрая != слиянием ведь, да?",2024-03-03 09:59:25,NaN
2,118190,423515.0,2024-03-03 09:49:53,user,Еще не проходили; 2024-03-03T10:01:13.790Z,2024-03-03 10:01:13,NaN


In [26]:
df['ID урока'] = df['ID урока'].apply(lambda x: int(x))

In [33]:
df_f = df[df['ID урока'] == 425512]

In [35]:
text = df_f['Текст сообщения'].tolist()
full_text = '\n'.join(x for x in text)

In [36]:
tokenizer = AutoTokenizer.from_pretrained('IlyaGusev/saiga_llama3_8b')
text_splitter = RecursiveCharacterTextSplitter.from_huggingface_tokenizer(
    tokenizer=tokenizer,
    chunk_size = 4096,  # 768
    chunk_overlap = 256,
    add_start_index = True,
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [37]:
chunks = text_splitter.create_documents([full_text])
chunks = [item.page_content for item in chunks]

In [38]:
definition_retrieval_system = """Ты — Сайга, русскоязычный автоматический ассистент. Ты разговариваешь с людьми и помогаешь им.""".strip()

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", definition_retrieval_system),
        ("human", BASE_PROMPT)
    ]
)

gpu_chain = prompt | gpu_llm.bind(stop=[" bot"]) 

In [39]:
prompts = [EXTRACT_INTRO_PROMPT % chunk for chunk in chunks]
questions = [{'prompt': cur_text} for cur_text in prompts]

In [ ]:
intro = gpu_chain.batch(questions)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [ ]:
print(intro[0])